<a href="https://colab.research.google.com/github/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install mmcv
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

Cloning into 'mmdetection'...
remote: Enumerating objects: 21781, done.
remote: Total 21781 (delta 0), reused 0 (delta 0), pack-reused 21781
Receiving objects: 100% (21781/21781), 25.19 MiB | 9.86 MiB/s, done.
Resolving deltas: 100% (15305/15305), done.
/home/ueidarchen/Homework 2/mmdetection
Obtaining file:///home/ueidarchen/Homework%202/mmdetection
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmdet


In [2]:
# Download the training data
!gdown https://drive.google.com/u/1/uc?id=1lrKueI4HrySQDGvpkilQN9BfaMUN7hZi
!unzip -q train.zip

Downloading...
From: https://drive.google.com/u/1/uc?id=1lrKueI4HrySQDGvpkilQN9BfaMUN7hZi
To: /home/ueidarchen/Homework 2/mmdetection/train.zip
100%|█████████████████████████████████████████| 420M/420M [00:01<00:00, 240MB/s]


In [3]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.10.0+cu102 True
2.18.1
10.2
GCC 7.3


## Perform inference with a MMDet detector
MMDetection already provides high level APIs to do inference and training.

In [4]:
# Mat file preprocessing
import h5py, json
import numpy as np
from PIL import Image

DATA_DIR = 'train'

mat = h5py.File(f'{DATA_DIR}/digitStruct.mat', 'r')
bboxes = mat['digitStruct/bbox']
names = mat['digitStruct/name']

In [ ]:
# Get dict list from mat file

KEYS = ['label', 'left', 'top', 'width', 'height']
ann_list = []

for bbox, name in zip(bboxes, names):
    filename = ''.join(chr(char[0]) for char in mat[name[0]][()])
    img = Image.open(f'{DATA_DIR}/{filename}')
    width, height = img.size
    if mat[bbox[0]][KEYS[0]].shape[0] > 1:
        attr = np.array([[mat[mat[bbox[0]][key][k][0]][0][0]
                          for k in range(mat[bbox[0]][key].shape[0])]
                             for key in KEYS]).T.astype(int)
    else:
        attr = np.array([mat[bbox[0]][key][0] for key in KEYS]).T.astype(int)
    labels = (attr[:, 0] % 10).tolist()
    bboxes_data = attr[:, 1:].tolist()
    obj = dict(
        filename = filename,
        width = width,
        height = height,
        ann = dict(
            bboxes = bboxes_data,
            labels = labels
        )
    )
    ann_list.append(obj)

In [ ]:
# Export dict list to JSON

import random, json

DATA_SIZE = len(ann_list)
TRAIN_RATIO = 0.7
TRAIN_SIZE = int(DATA_SIZE * TRAIN_RATIO)
VAL_SIZE = DATA_SIZE - TRAIN_SIZE

img_all = [ann['filename'] for ann in ann_list]
random.shuffle(img_all)
img_train = set(img_all[: TRAIN_SIZE])
img_val = set(img_all[TRAIN_SIZE: ])
ann_train = [ann for ann in ann_list if ann['filename'] in img_train]
ann_val = [ann for ann in ann_list if ann['filename'] in img_val]

with open(f'{DATA_DIR}/ann_train.json', 'w') as f:
    f.write(json.dumps(ann_train))
with open(f'{DATA_DIR}/ann_val.json', 'w') as f:
    f.write(json.dumps(ann_val))

In [ ]:
# Define and register custom dataset

import os.path as osp
import copy

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

@DATASETS.register_module()
class DigitDataset(CustomDataset):
    CLASSES = tuple(str(i) for i in range(10))
    def load_annotations(self, ann_file):
        ann_list = mmcv.load(self.ann_file)
        for i, a in enumerate(ann_list):
            ann_list[i]['ann']['bboxes'] = np.array(
                ann_list[i]['ann']['bboxes'], dtype=np.float32)
            # (x, y, w, h) -> #(xmin, ymin, xmax, ymax)
            ann_list[i]['ann']['bboxes'][:, 2:] += ann_list[i]['ann']['bboxes'][:, :2] - 1
            ann_list[i]['ann']['labels'] = np.array(
                ann_list[i]['ann']['labels'], dtype=np.long)
        return ann_list

In [ ]:
# Download Pretrained weights
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth -O checkpoints/faster_rcnn.pth

In [ ]:
# Import Configuration File
from mmcv import Config
cfg = Config.fromfile('../config.py')
print(cfg.pretty_text)

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Train detector
train_detector(model, datasets, cfg, distributed=False, validate=True)